In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re
import requests
import bs4
import numpy as np
import time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
YEAR_START = 2021 
YEAR_END = 2021

# the path you want to save the result
filepath = "race_result_2021season_git.csv"

In [3]:
# Set up selenium to use Firefox
options = Options()
options.headless = True #No need to open a browser window

# Example of manaully specifying the WebDriver's location: 
driver = webdriver.Firefox(executable_path="../Others/geckodriver.exe",options=options) #Windows

In [4]:
def table(url, date, match='1'): # default set match = 1 for checking if it is a race date only
    
    driver.get(url)
    
    # Is there anything?
    #if driver.page_source.find("No information.") != -1:
        #return []
    
    # Wait 10 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(((By.XPATH, 
                                             '//table[@class="f_tac table_bd draggable"]'))))
    except Exception as e:
        print(e, date, match)
        return []
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # main table
    table = soup.find('table', class_ = 'f_tac table_bd draggable')
    
    # Class, Distance, Rating table
    cdr_table = soup.find('tbody', class_ = 'f_fs13')
    
    # for cdr data
    cdr_list = []
    # for main table data
    output_list = []
    
    #horseid
    horse_id = soup.find_all(href=re.compile("HorseId"))
    horse_id_list = []
    for horse in soup.find_all(href=re.compile("HorseId")):
        horse_id = horse['href'].split('HorseId=')[1]
        horse_id_list.append([horse_id])
    
    
    
    # fetchin cdr table
    for cdr_row in cdr_table.find_all('tr')[1:4:2]:    # First table row(0) is blank
        # Second table row(1) is cdr, Fourth table row(3) is Prize Money
        cdr_cols = cdr_row.find_all('td')
        cdr_cols = [ele.text.strip() for ele in cdr_cols]
        
        # it stores both cdr and prize money
        cdr_list.append(cdr_cols[0])
        
    # fetching main table
    rows = table.find('tbody').find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip()
                .replace('\n','') 
                .replace(' '*20,' ')
                .replace("-", " ") for ele in cols]
        
        
        cols.append(str(date))
        cols.append(str(match))
        
        # extend cdr list to the main list
        cols.extend(cdr_list)
        output_list.append(cols)
    
    
    data = np.concatenate([output_list, horse_id_list], axis=1)
    
    return data

In [5]:
url_front = "https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate="

import csv
with open(filepath, 'w', newline='') as csvfile:
    mywriter = csv.writer(csvfile)
    mywriter.writerow(['Plc', 'Horse No', 'Horse', 'Jockey', 'Trainer', 'ActualWt',
                       'Declar_HorseWt', 'Draw', 'LBW', 'RunningPosition', 'FinishTime', 
                       'Win_Odds', 'date', 'match', 'cdr', 'prize_money', 'horseid'])
    #Copy the loop from above and incorporate the csv-saving code
    for year in range(YEAR_START, YEAR_END+1):
        for month in range(1, 13):
            start = time.time()
            for day in range(1, 32):
                
                #Convert month and day to 2-digit representation
                month_2d = '{:02d}'.format(month)
                day_2d = '{:02d}'.format(day)
                
                #Full URL of data source
                date = str(year) + '/' + month_2d + '/' + day_2d
                url = url_front + date
                
                #check if is a race day
                check = table(url, date)
                #content = np.array(content)
                
                # Proceed only save if there is something in content
                if len(check) > 0:
                    #print("{} is a race day".format(date))
                    
                    for match in range(1,13):
                        time.sleep(5)
                        
                        url = url_front + date + '&RaceNo=' + str(match)
                        
                        content = table(url, date, match) # what really write into the csv
                        #content = np.array(content)
                        if len(content) > 0:
                            mywriter.writerows(content)
                            
            print(year, month, (time.time() - start)/60)
            
driver.close()

Message: 
 2021/03/01 1
Message: 
 2021/03/02 1
2021/03/03 is a race day
Message: 
 2021/03/03 10
Message: 
 2021/03/03 11
Message: 
 2021/03/03 12
Message: 
 2021/03/04 1
Message: 
 2021/03/05 1
Message: 
 2021/03/06 1
2021/03/07 is a race day
Message: 
 2021/03/07 11
Message: 
 2021/03/07 12
Message: 
 2021/03/08 1
Message: 
 2021/03/09 1
2021/03/10 is a race day
Message: 
 2021/03/10 10
Message: 
 2021/03/10 11
Message: 
 2021/03/10 12
Message: 
 2021/03/11 1
Message: 
 2021/03/12 1
2021/03/13 is a race day
Message: 
 2021/03/13 11
Message: 
 2021/03/13 12
Message: 
 2021/03/14 1
Message: 
 2021/03/15 1
Message: 
 2021/03/16 1
2021/03/17 is a race day
Message: 
 2021/03/17 10
Message: 
 2021/03/17 11
Message: 
 2021/03/17 12
Message: 
 2021/03/18 1
Message: 
 2021/03/19 1
Message: 
 2021/03/20 1
2021/03/21 is a race day
Message: 
 2021/03/21 11
Message: 
 2021/03/21 12
Message: 
 2021/03/22 1
Message: 
 2021/03/23 1
2021/03/24 is a race day
Message: 
 2021/03/24 10
Message: 
 2021/0